<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [1]:
! pip install torchtext==0.8.1

     |████████████████████████████████| 7.0 MB 8.6 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [8]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [12]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 30.6MB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [14]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True, # `rnn.pack_padded_sequence` requires that a minibatch be sorted by decreasing order, 
    device = device)          # which requires reversing relative to typical sort keys
    

# BucketIterator - 
# Defines an iterator that batches examples of similar lengths together.
# Minimizes amount of padding needed while producing freshly shuffled batches for each new epoch. 
# See pool for the bucketing procedure used.

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
next(iter(train_iter))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)



[torchtext.data.batch.Batch of size 64]
	[.text]:('[torch.cuda.LongTensor of size 113x64 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')
	[.label]:[torch.cuda.FloatTensor of size 64 (GPU 0)]

In [ ]:
txt, lab = next(iter(train_iter))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
txt

# Первый тензор: индексы слов в словаре для каждого примера в batch - text
# Второй тензор: Длины каждого примера в batch - text_lengths
# Уже с паддингом - 1 в конце

(tensor([[  8934,  16364,      9,  ..., 168225,     10,    303],
         [   279,   1114,    102,  ...,  40953,     24,    712],
         [    27,    272,   1912,  ...,   8805,   2074,     43],
         ...,
         [     8,    653,    622,  ...,      1,      1,      1],
         [     2,    125,      2,  ...,      1,      1,      1],
         [  2800,    677,  14862,  ...,      1,      1,      1]],
        device='cuda:0'),
 tensor([234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234, 234,
         234, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 232,
         232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
         232, 231, 231, 231, 231, 231, 231, 231, 231, 231, 231, 231, 231, 230,
         230, 230, 230, 230, 230, 230, 230, 230], device='cuda:0'))

In [ ]:
txt[0].shape # [sent len, batch size]

torch.Size([234, 64])

In [ ]:
lab
# Третий тензор: label для каждого примера в batch.

tensor([0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 1., 0., 0., 0., 1., 0., 1.], device='cuda:0')

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

1. Сначала используем hidden_state для классификации

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # pad_idx - номер вектора-паддинга, для нового эмбеддинга инициализируется нулями, 
        # не влияет на градиент, т.е. не обновляется во время обучения
        # нужен для выравнивания элементов в батче до одной длины
        
        # self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim,
        #                   num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                          num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(dropout)
        self.elu = nn.ELU()

        if bidirectional:
          # конкатенация векторов от прохода слева направо и от прохода справа налево
          self.fc = nn.Linear(hidden_dim*2, output_dim) 
        else:  
          self.fc = nn.Linear(hidden_dim, output_dim)
        
        
    def forward(self, text, text_lengths):

        # sent len - длина предложения с паддингом
        # text_lengths - длина предложения без паддинга
        # вход - индексы токенов, выход - эмбединги токенов, 
        # инициализируются из N (0, 1), содержат вектора-паддинги
        # Dim transformation: text = [sent len, batch size] -> embedded = [sent len, batch size, emb dim]
        embedded = self.embedding(text) # [sent len, 64, 100]
        embedded = self.dropout(embedded)

        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # nn.utils.rnn.pack_padded_sequence - 
        # - Packs a Tensor containing padded sequences of variable length
        # text_lengths - list of sequence lengths of each batch element
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        _, (hidden, cell) = self.rnn(packed_embedded)
        
        #hidden = [num layers * num directions, batch size, hid dim] [3*2=6, 64, 256]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        if bidirectional:
          hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # [64, 512]   
        else:
          hidden = hidden[-1, :, :] # [64, 256]  
        #hidden = [batch size, hid dim * num directions] 

        # hidden = self.elu(hidden)
        hidden = self.dropout(hidden)

        return self.fc(hidden)

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 300
hidden_dim = 256
output_dim = 1
n_layers = 4
bidirectional = True
dropout = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # индекс строки-паддинга!
# text.vocab.stoi – A collections.defaultdict instance mapping token strings to numerical identifiers
# text.pad_token – The string token used as padding. Default: “<pad>”
patience=4

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.005)
loss_func = nn.BCEWithLogitsLoss()
# scheduler = torch.optim.lr_scheduler.StepLR(opt, 2, 0.5)

max_epochs = 20

In [3]:
# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354

def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. 0 <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    
    '''
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):

    train_loss = 0.0
    train_f1 = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        opt.zero_grad()

        text_embedding = batch.text[0].to(device)
        text_lengths = batch.text[1].cpu()
        targets = torch.unsqueeze(batch.label, 1).to(device)

        preds = model(text_embedding, text_lengths)
        loss = loss_func(preds, targets) 
        loss.backward()
        opt.step()
        train_loss += loss

        preds = torch.sigmoid(preds)
        preds = (preds>0.5).type(torch.long).view(-1)
        targets = targets.view(-1)
        train_f1 += f1_loss(targets, preds)

    train_loss /= len(train_iter) 
    train_f1 /= len(train_iter) 
    # scheduler.step()

    val_loss = 0.0
    val_f1 = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    with torch.no_grad():

      for it, batch in pbar:
          # YOUR CODE GOES HERE

          text_embedding = batch.text[0].to(device)
          text_lengths = batch.text[1].cpu()
          targets = torch.unsqueeze(batch.label, 1).to(device)

          preds = model(text_embedding, text_lengths)
          loss = loss_func(preds, targets) 
          val_loss += loss

          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1)
          targets = targets.view(-1)
          val_f1 += f1_loss(targets, preds)

    val_loss /= len(valid_iter)
    val_f1 /= len(valid_iter) 

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training F1: {}, \
    Validation F1: {}'.format(epoch, train_loss, val_loss, train_f1, val_f1))
model.load_state_dict(best_model)

  0%|          | 0/274 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6942651867866516, Validation Loss: 0.6716118454933167, Training F1: 0.4799090623855591,     Validation F1: 0.6881729960441589


  0%|          | 0/274 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 6, Training Loss: 0.1895250529050827, Validation Loss: 0.4287640154361725, Training F1: 0.9292290210723877,     Validation F1: 0.8625495433807373


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

<All keys matched successfully>

In [4]:
# f1 на тесте на всем датасете с использованием f1_score из sklearn.metrics

from sklearn.metrics import f1_score

In [ ]:
preds_list = []
targets_list = []

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)

with torch.no_grad():
   for it, batch in pbar:

          text_embedding = batch.text[0].to(device)
          text_lengths = batch.text[1].cpu()

          preds = model(text_embedding, text_lengths)
          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1).cpu()
          preds = preds.tolist()
          preds_list.append(preds)

          targets = batch.label.tolist()
          targets_list.append(targets)

  0%|          | 0/391 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
targets_list_ = [y for x in targets_list for y in x]
preds_list_ = [y for x in preds_list for y in x]

f1_score(targets_list_, preds_list_)

0.8510200190657768

In [ ]:
# RNN

# emb_dim = 200, hidden_dim = 128, StepLR(opt, 1, 0.5)
# 0.7556562854128405 - dropout in RNN, dropout = 0.5, patience=4
# 0.7367893395714107 - StepLR(opt, 2, 0.5)
# 0.7404932263102458 - AdamW + StepLR(opt, 2, 0.5)
# 0.6959295648845221 - AdamW без scheduler
# 0.7552816187502533 - emb_dim = 300, hidden_dim = 256, StepLR(opt, 1, 0.5), dropout in RNN, dropout = 0.5, patience=4

# LSTM

# emb_dim = 200, hidden_dim = 128, StepLR(opt, 1, 0.5)
# 0.8383966072666476 - dropout in RNN, dropout = 0.5, patience=4, Adam без scheduler 
# 0.8286779178357554 - dropout in RNN, dropout = 0.5, patience=4, Adam без scheduler, embedded = self.dropout(embedded)
# 0.8193021900241485 - dropout in RNN, dropout = 0.5, patience=4, Adam c scheduler
# 0.7894822906105896 - dropout in RNN, dropout = 0.5, patience=4, AdamW

# emb_dim = 300, hidden_dim = 256, dropout in RNN, dropout = 0.5, patience=4, Adam без scheduler 
# 0.850388506084152 - embedded = self.dropout(embedded) 
# 0.8260347930413916 - nn.ELU()
# 0.8510200190657768 - n_layers = 4 - max!
# 0.7805744218049453 - Adam с scheduler 

Максимальное значеие f1 при использовании hidden - 0.85. LSTM работает лучше, чем RNN.

Теперь используем output для классификации.

In [ ]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # pad_idx - номер вектора-паддинга, для нового эмбеддинга инициализируется нулями, 
        # не влияет на градиент, т.е. не обновляется во время обучения
        # нужен для выравнивания элементов в батче до одной длины
        
        # self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim,
        #                   num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                          num_layers=n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)

        if bidirectional:
          # конкатенация векторов от прохода слева направо и от прохода справа налево
          self.fc = nn.Linear(hidden_dim*2, output_dim) 
        else:  
          self.fc = nn.Linear(hidden_dim, output_dim)

        # if bidirectional:
        #   # конкатенация векторов от прохода слева направо и от прохода справа налево
        #   self.fc_1 = nn.Linear(hidden_dim*2, 64) 
        #   self.fc_2 = nn.Linear(64, output_dim) 
        # else:  
        #   self.fc_1 = nn.Linear(hidden_dim, 64) 
        #   self.fc_2 = nn.Linear(64, output_dim) 
        
        
    def forward(self, text, text_lengths):

        # sent len - длина предложения с паддингом
        # text_lengths - длина предложения без паддинга
        # вход - индексы токенов, выход - эмбединги токенов, 
        # инициализируются из N (0, 1), содержат вектора-паддинги
        # Dim transformation: text = [sent len, batch size] -> embedded = [sent len, batch size, emb dim]
        embedded = self.embedding(text) # [sent len, 64, 100]
        # embedded = embedded.permute(1, 0, 2) # [64, sent len, 100]
        embedded = self.dropout(embedded)

        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # nn.utils.rnn.pack_padded_sequence - 
        # - Packs a Tensor containing padded sequences of variable length
        # text_lengths - list of sequence lengths of each batch element
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # packed_output, hidden = self.rnn(packed_embedded)

        #unpack sequence
        # https://newbedev.com/why-do-we-pack-the-sequences-in-pytorch
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions] [sent len, 64, 64 * 2 = 128]
        #output over padding tokens are zero tensors

        output = self.dropout(output)

        if bidirectional:
        # https://towardsdatascience.com/lstm-text-classification-using-pytorch-2c6c657f8fc0
        # берем последний выход сети, соответствующий последнему реальному слову (не паддингу!): [sent len, 64, 128] -> [64, 128]  
          out_forward = output[output_lengths-1, range(output_lengths.size(0)), :hidden_dim]  
          out_reverse = output[0, :, hidden_dim:]
          out = torch.cat((out_forward, out_reverse), 1)
        else:
          out = output[output_lengths-1, range(output_lengths.size(0)), :]  #[64, 128] 


        return self.fc(out)  # [64, 1]

        # out = self.fc_1(out)
        # return self.fc_2(out)  

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 200
hidden_dim = 128
# emb_dim = 300
# hidden_dim = 256
output_dim = 1
n_layers = 3
# bidirectional = True
bidirectional = False
dropout = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # индекс строки-паддинга!
# text.vocab.stoi – A collections.defaultdict instance mapping token strings to numerical identifiers
# text.pad_token – The string token used as padding. Default: “<pad>”
patience=4

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.006)
loss_func = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.StepLR(opt, 2, 0.5)

max_epochs = 20

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):

    train_loss = 0.0
    train_f1 = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        opt.zero_grad()

        text_embedding = batch.text[0].to(device)
        text_lengths = batch.text[1].cpu()
        targets = torch.unsqueeze(batch.label, 1).to(device)

        preds = model(text_embedding, text_lengths)
        loss = loss_func(preds, targets) 
        loss.backward()
        opt.step()
        train_loss += loss

        preds = torch.sigmoid(preds)
        preds = (preds>0.5).type(torch.long).view(-1)
        targets = targets.view(-1)
        train_f1 += f1_loss(targets, preds)

    train_loss /= len(train_iter) 
    train_f1 /= len(train_iter) 
    scheduler.step()

    val_loss = 0.0
    val_f1 = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    with torch.no_grad():

      for it, batch in pbar:
          # YOUR CODE GOES HERE

          text_embedding = batch.text[0].to(device)
          text_lengths = batch.text[1].cpu()
          targets = torch.unsqueeze(batch.label, 1).to(device)

          preds = model(text_embedding, text_lengths)
          loss = loss_func(preds, targets) 
          val_loss += loss

          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1)
          targets = targets.view(-1)
          val_f1 += f1_loss(targets, preds)

    val_loss /= len(valid_iter)
    val_f1 /= len(valid_iter) 

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training F1: {}, \
    Validation F1: {}'.format(epoch, train_loss, val_loss, train_f1, val_f1))
model.load_state_dict(best_model)

  0%|          | 0/274 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6578109264373779, Validation Loss: 0.5520431995391846, Training F1: 0.5955792665481567,     Validation F1: 0.7653595805168152


  0%|          | 0/274 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 6, Training Loss: 0.1887199431657791, Validation Loss: 0.4366251528263092, Training F1: 0.9266526699066162,     Validation F1: 0.8628040552139282


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

<All keys matched successfully>

In [ ]:
preds_list = []
targets_list = []

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)

with torch.no_grad():
   for it, batch in pbar:

          text_embedding = batch.text[0].to(device)
          text_lengths = batch.text[1].cpu()

          preds = model(text_embedding, text_lengths)
          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1).cpu()
          preds = preds.tolist()
          preds_list.append(preds)

          targets = batch.label.tolist()
          targets_list.append(targets)

  0%|          | 0/391 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
targets_list_ = [y for x in targets_list for y in x]
preds_list_ = [y for x in preds_list for y in x]

f1_score(targets_list_, preds_list_)

0.8571541314813353

In [ ]:
# output, bidirectional = False

# 0.7156312105763415 
# 0.8003960956288018 - StepLR(opt, 2, 0.5)
# 0.8318155947638018 - lr=0.003, StepLR(opt, 2, 0.5)
# 0.8489060700773153 - lr=0.005, StepLR(opt, 2, 0.5)

# 0.8496678966789668 - lr=0.005, emb_dim = 200, hidden_dim = 128
# 0.834866663840251  - lr=0.005, emb_dim = 300, hidden_dim = 256
# 0.8042833753705916 - lr=0.006, emb_dim = 200, hidden_dim = 128
# 0.84895110143731 - lr=0.005, emb_dim = 200, hidden_dim = 128, self.fc_1/2
# 0.7997642744892612 - lr=0.007, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5)
# 0.8571541314813353 - lr=0.006, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5)

# output, bidirectional = True

# 0.8494250775688993 - lr=0.005, emb_dim = 200, hidden_dim = 128
# 0.3054646324549237 - lr=0.005, emb_dim = 300, hidden_dim = 256 СЕТЬ НЕ УЧИЛАСЬ
# 0.7254731724257684 - lr=0.005, emb_dim = 300, hidden_dim = 256, self.fc_1/2
# 0.8157468159011965 - lr=0.006, emb_dim = 200, hidden_dim = 128
# 0.8498107557293183 - lr=0.005, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5)
# 0.8586947965047627 - lr=0.006, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5)
# 0.6625485541648685 - lr=0.006, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5), out = self.dropout(out)  СЕТЬ НЕ УЧИЛАСЬ
# 0.8554649265905384 - lr=0.006, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5), self.fc_1/2
# 0.6666666666666666 - lr=0.006, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5), self.fc_1/2 и dropout между ними СЕТЬ НЕ УЧИЛАСЬ
# 0.8254340277777777 - lr=0.007, emb_dim = 200, hidden_dim = 128, StepLR(opt, 2, 0.5)

При использовании для классификации output, вместо hidden, удалось чуть повысить качество, с 0.85 до 0.86

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**: 0.86

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

https://habr.com/ru/company/ods/blog/353060/ - Применение сверточных нейронных сетей для задач NLP

In [17]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [18]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256), # сколько объектов в батче в trn, vld, tst
        sort=False,
        # sort_key= lambda x: len(x.src), 
        sort_within_batch=False,
        device=device,
        repeat=False
)

# https://pytorch.org/text/_modules/torchtext/data/iterator.html

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
next(iter(train_iter))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)



[torchtext.data.batch.Batch of size 128]
	[.text]:[torch.cuda.LongTensor of size 128x834 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 128 (GPU 0)]

In [ ]:
txt, lab = next(iter(train_iter))
txt # индексы слов в словаре для каждого примера в batch - text

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


tensor([[  2385,   1794,   7918,  ...,      1,      1,      1],
        [    15,      3,     20,  ...,      1,      1,      1],
        [  8952,   5646, 147864,  ...,      1,      1,      1],
        ...,
        [     9,     83,    118,  ...,      1,      1,      1],
        [     9,    408,    198,  ...,      1,      1,      1],
        [   191,    121,   3326,  ...,      1,      1,      1]],
       device='cuda:0')

In [ ]:
txt.shape, lab.shape

(torch.Size([128, 898]), torch.Size([128]))

In [ ]:
lab

tensor([0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
        1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
        1., 1.], device='cuda:0')

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [22]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
 
        self.conv_0 = nn.Conv2d(1, out_channels, (kernel_sizes[0], emb_dim)) # YOUR CODE GOES HERE
        
        self.conv_1 = nn.Conv2d(1, out_channels, (kernel_sizes[1], emb_dim))  # YOUR CODE GOES HERE
        
        self.conv_2 = nn.Conv2d(1, out_channels, (kernel_sizes[2], emb_dim))  # YOUR CODE GOES HERE

        self.conv_3 = nn.Conv2d(1, out_channels, (kernel_sizes[3], emb_dim)) 

        # self.conv_4 = nn.Conv2d(1, out_channels, (kernel_sizes[4], emb_dim)) 
        
        # self.fc_0 = nn.Linear(len(kernel_sizes) * out_channels, out_channels)
        # self.fc_1 = nn.Linear(out_channels, 1)

        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.norm = nn.LayerNorm(emb_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.embedding(text) # [128, 967, 300]
        embedded = self.norm(embedded)
        embedded = self.dropout(embedded)
        # сделаем число каналов = 1 для nn.Conv2d, у которого input size (N, C, H, W)
        embedded = embedded.unsqueeze(1) # [128, 1, 967, 300] # may be reshape here

        # self.conv_0(embedded) - [128, 64, 795, 1] -> .squeeze(3)
        # уберем лишнюю размерность для F.max_pool1d, у которого input size (N, C, L)
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3)) # [128, 64, 795] # may be reshape here 
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))  # may be reshape here
        conved_3 = F.relu(self.conv_3(embedded).squeeze(3)) 
        # conved_4 = F.relu(self.conv_4(embedded).squeeze(3)) 
        
        # F.max_pool1d(conved_0, conved_0.shape[2]) - [128, 64, 1]) -> .squeeze(2)
        #  уберем лишнюю размерность для torch.cat
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2) # [128, 64]
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)
        # pooled_4 = F.max_pool1d(conved_4, conved_4.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim=1))

        return self.fc(cat)

        # out = self.fc_0(cat)
        # return self.fc_1(out)

In [ ]:
kernel_sizes = [2, 3, 4, 5]
# kernel_sizes = [2, 3, 4]
vocab_size = len(TEXT.vocab)
out_channels = 64
dropout = 0.5
emb_dim = 300

patience=4

model = CNN(vocab_size=vocab_size, emb_dim=emb_dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

model = model.to(device)

In [ ]:
lr_cnn = 4e-3
opt = torch.optim.Adam(model.parameters(), lr = lr_cnn)
loss_func = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.StepLR(opt, 3, 0.5)

max_epochs = 30

Обучите!

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    train_f1 = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        opt.zero_grad()

        text_embedding = batch.text.to(device)
        targets = torch.unsqueeze(batch.label, 1).to(device)

        preds = model(text_embedding)
        loss = loss_func(preds, targets) 
        loss.backward()
        opt.step()
        train_loss += loss

        preds = torch.sigmoid(preds)
        preds = (preds>0.5).type(torch.long).view(-1)
        targets = targets.view(-1)
        train_f1 += f1_loss(targets, preds)

    train_loss /= len(train_iter) 
    train_f1 /= len(train_iter) 
    scheduler.step()

    val_loss = 0.0
    val_f1 = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    with torch.no_grad():

      for it, batch in pbar:
          # YOUR CODE GOES HERE

          text_embedding = batch.text.to(device)
          targets = torch.unsqueeze(batch.label, 1).to(device)

          preds = model(text_embedding)
          loss = loss_func(preds, targets) 
          val_loss += loss

          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1)
          targets = targets.view(-1)
          val_f1 += f1_loss(targets, preds)

    val_loss /= len(val_iter)
    val_f1 /= len(val_iter) 

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training F1: {}, \
    Validation F1: {}'.format(epoch, train_loss, val_loss, train_f1, val_f1))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.7537987232208252, Validation Loss: 0.5192634463310242, Training F1: 0.5827876925468445,     Validation F1: 0.7182818055152893


  0%|          | 0/137 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 18, Training Loss: 0.12373773753643036, Validation Loss: 0.31196466088294983, Training F1: 0.9535521268844604,     Validation F1: 0.8719927668571472


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

In [ ]:
preds_list = []
targets_list = []

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)

with torch.no_grad():
   for it, batch in pbar:

          text_embedding = batch.text.to(device)

          preds = model(text_embedding)
          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1).cpu()
          preds = preds.tolist()
          preds_list.append(preds)

          targets = batch.label.tolist()
          targets_list.append(targets)

  0%|          | 0/98 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
targets_list_ = [y for x in targets_list for y in x]
preds_list_ = [y for x in preds_list for y in x]

f1_score(targets_list_, preds_list_)

0.8737871767454755

In [ ]:
# 0.857 kernel_sizes = [3, 4, 5]
# 0.8347996089931573 - kernel_sizes = [2, 3, 4]
# 0.8723547861992297 - kernel_sizes = [3, 4, 5]
# 0.8750397709195038 - kernel_sizes = [2, 3, 4, 5], r_cnn = 2e-3
# 0.8311649580615097 - kernel_sizes = [2, 3, 4, 5], lr_cnn = 2e-3, StepLR(opt, 3, 0.5)
# 0.8721235521235521 - kernel_sizes = [2, 3, 4, 5], lr_cnn = 3e-3

# 0.8711544319220226 - kernel_sizes = [2, 3, 4, 5],  lr_cnn = 3e-3, # sort=False
# 0.874564723694171 - kernel_sizes = [2, 3, 4, 5], lr_cnn = 3e-3, ELU
# 0.8709467696058514 - kernel_sizes = [2, 3, 4, 5, 6], lr_cnn = 3e-3,
# 0.8693883815516845- kernel_sizes = [2, 3, 4, 5], lr_cnn = 3e-3, fc_0/fc_1
# 0.8737871767454755 - kernel_sizes = [2, 3, 4, 5], lr_cnn = 4e-3, StepLR(opt, 3, 0.5)

C CNN удалось еще улучшить качество, по сравнению с RNN - с 0.86 до 0.87

Посчитайте f1-score вашего классификатора.

**Ответ**: 0.87

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [5]:
!pip install -q captum

     |████████████████████████████████| 1.4 MB 8.7 MB/s 


In [103]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [104]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.81 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.47 ) , delta:  tensor([1.0490e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.64 ) , delta:  tensor([2.6030e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.08 ) , delta:  tensor([5.9279e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.25 ) , delta:  tensor([6.2562e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.50 ) , delta:  tensor([1.3436e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.81),pos,1.74,It was a fantastic performance ! pad
pos,neg (0.47),pos,0.27,Best film ever pad pad pad pad
pos,pos (0.64),pos,1.42,Such a great show! pad pad pad
neg,neg (0.08),pos,-0.74,It was a horrible movie pad pad
neg,neg (0.25),pos,-0.62,I've never watched something as bad pad
neg,neg (0.50),pos,0.83,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.81),pos,1.74,It was a fantastic performance ! pad
pos,neg (0.47),pos,0.27,Best film ever pad pad pad pad
pos,pos (0.64),pos,1.42,Such a great show! pad pad pad
neg,neg (0.08),pos,-0.74,It was a horrible movie pad pad
neg,neg (0.25),pos,-0.62,I've never watched something as bad pad
neg,neg (0.50),pos,0.83,It is a disgusting movie! pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [19]:
from torchtext.vocab import Vectors, GloVe

TEXT.build_vocab(trn, vectors=GloVe())# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [2, 3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

patience=4

.vector_cache/glove.840B.300d.zip: 2.18GB [06:50, 5.30MB/s]                            
100%|█████████▉| 2196016/2196017 [05:23<00:00, 6778.60it/s]


In [21]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        # sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [98]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

# word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

# model.embedding.weight = model.embedding.weight.data.copy_(word_embeddings) # инициализируйте эмбэдинги 
model.embedding.weight = torch.nn.Parameter(word_embeddings) 

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters(), lr=2e-3) # 

In [99]:
max_epochs = 30
loss_func = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.StepLR(opt, 3, 0.5)

Попробуем заморозить эмбединги на первых итерациях обучения. Это поможет не сильно портить наши эмбединги на первых итерациях.

In [25]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embedding
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

Вы знаете, что делать.

In [100]:
min_loss = np.inf

cur_patience = 0

num_iter = 0
num_freeze_iter=1500
freeze_embeddings(model)

for epoch in range(1, max_epochs + 1):
    
    train_loss = 0.0
    train_f1 = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        if num_iter > num_freeze_iter:
            freeze_embeddings(model, True)
        #YOUR CODE GOES HERE
        opt.zero_grad()

        text_embedding = batch.text.to(device)
        targets = torch.unsqueeze(batch.label, 1).to(device)

        preds = model(text_embedding)
        loss = loss_func(preds, targets) 
        loss.backward()
        opt.step()
        num_iter += 1
        train_loss += loss

        preds = torch.sigmoid(preds)
        preds = (preds>0.5).type(torch.long).view(-1)
        targets = targets.view(-1)
        train_f1 += f1_loss(targets, preds)

    train_loss /= len(train_iter) 
    train_f1 /= len(train_iter) 
    scheduler.step()

    val_loss = 0.0
    val_f1 = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    with torch.no_grad():

      for it, batch in pbar:
          # YOUR CODE GOES HERE

          text_embedding = batch.text.to(device)
          targets = torch.unsqueeze(batch.label, 1).to(device)

          preds = model(text_embedding)
          loss = loss_func(preds, targets) 
          val_loss += loss

          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1)
          targets = targets.view(-1)
          val_f1 += f1_loss(targets, preds)

    val_loss /= len(val_iter)
    val_f1 /= len(val_iter) 

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Training F1: {}, \
    Validation F1: {}'.format(epoch, train_loss, val_loss, train_f1, val_f1))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.5705353617668152, Validation Loss: 0.3676181137561798, Training F1: 0.6929643750190735,     Validation F1: 0.8425884246826172


  0%|          | 0/137 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 10, Training Loss: 0.2801569402217865, Validation Loss: 0.3028746545314789, Training F1: 0.8819966316223145,     Validation F1: 0.8724501132965088


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

In [101]:
preds_list = []
targets_list = []

model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)

with torch.no_grad():
   for it, batch in pbar:

          text_embedding = batch.text.to(device)

          preds = model(text_embedding)
          preds = torch.sigmoid(preds)
          preds = (preds>0.5).type(torch.long).view(-1).cpu()
          preds = preds.tolist()
          preds_list.append(preds)

          targets = batch.label.tolist()
          targets_list.append(targets)

  0%|          | 0/98 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [102]:
targets_list_ = [y for x in targets_list for y in x]
preds_list_ = [y for x in preds_list for y in x]

f1_score(targets_list_, preds_list_)

0.8760493827160495

In [ ]:
# freeze_embeddings, lr=2e-3

# 0.8144642640456256 - num_freeze_iter=137 (ровно 1 эпоха), переобучается на 3 эпохе
# 0.8224345045303596 - num_freeze_iter=300
# 0.832620903528623 - num_freeze_iter=500
# 0.8577391885122725 - num_freeze_iter=800
# 0.8652083587926188 - num_freeze_iter=1000
# 0.8739659367396593 - num_freeze_iter=1200
# 0.8821608546663978 - num_freeze_iter=1500 - !!!
# 0.8757144645166673 - num_freeze_iter=1800
# 0.8755467524964926 - num_freeze_iter=2000

# 0.8640903880792532 - num_freeze_iter=1500, lr=1e-3
# 0.8711218175021805 - num_freeze_iter=1500, lr=3e-3
# 0.8742214672814042 - num_freeze_iter=1500, lr=3e-3, StepLR(opt, 3, 0.5)
# 0.8760493827160495 - num_freeze_iter=1500, lr=2e-3, StepLR(opt, 3, 0.5)

С предобученными эмбеддингами и заморозкой слоев качество еще немного повысилось, с 0.87 до 0.88

Посчитайте f1-score вашего классификатора.

**Ответ**: 0.88

Проверим насколько все хорошо!

In [105]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.88 ) , delta:  tensor([7.6980e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.49 ) , delta:  tensor([3.0544e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.66 ) , delta:  tensor([3.6856e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.12 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.40 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.25 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)


In [106]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.88),pos,1.32,It was a fantastic performance ! pad
pos,neg (0.49),pos,0.51,Best film ever pad pad pad pad
pos,pos (0.66),pos,1.42,Such a great show! pad pad pad
neg,neg (0.12),pos,-1.08,It was a horrible movie pad pad
neg,neg (0.40),pos,-0.86,I've never watched something as bad pad
neg,neg (0.25),pos,-0.79,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.88),pos,1.32,It was a fantastic performance ! pad
pos,neg (0.49),pos,0.51,Best film ever pad pad pad pad
pos,pos (0.66),pos,1.42,Such a great show! pad pad pad
neg,neg (0.12),pos,-1.08,It was a horrible movie pad pad
neg,neg (0.40),pos,-0.86,I've never watched something as bad pad
neg,neg (0.25),pos,-0.79,It is a disgusting movie! pad pad
